<a href="https://colab.research.google.com/github/alvumu/TGINE/blob/main/Practica1/3_2_scrapyJSON_LD_digitalTrends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sesión 3 - Scrapy - Ejemplo Crawling JSON-LD (ElMundo)

## Apartado 1.1 Crawler de El Mundo extrayendo JSON-LD
En el ejemplo siguiente definimos un crawler en Scrapy para extraer noticias de los metadatos de las páginas web.
El Mundo y otros periódicos publican metadatos en formato JSON-LD (https://json-ld.org/) que permite obtener información estructurada de las webs. En este caso, tendremos que obtener esos objetos JSON-LD y extraer su información en el formato de noticia (https://schema.org/NewsArticle) publicado en Schema.org

** Existen librerías de Python para trabajar directamente con JSON-LD

In [ ]:
!pip3 install -U scrapy

In [ ]:
import scrapy
import sys
import json
import locale
import time
import random

import hashlib

from bs4 import BeautifulSoup


# Para cada crawler que nos definimos nos debemos crear una clase Spider que debe heredar de la clase scrapy.Spider

class DigitalTrendsSpider (scrapy.Spider):
    name = 'DigitalTrends'

    # Decimos que el dominio válido es el de la UM
    allowed_domains = ['www.digitaltrends.com']

    # podemos definir las páginas de inicio
    start_urls = ['https://www.digitaltrends.com']

    # para evitar que el sitio te bloquee por usar scrapy es interesante cambiar el USER_AGENT
    # El user agent por defecto de Scrapy cuando hace una petición es
    # Scrapy/VERSION (+https://scrapy.org)
    custom_settings = {
        'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    }

    nDocumentos = 0
    visited_urls = set()

    # debemos de implementar este método que se llamará para cada una de las páginas que se vayan analizando
    def parse (self, response):
        """
        @inherit

        @param self
        @param response
        """

        if self.nDocumentos >= 1500:
          return

        # Guardamos la URL del sitio que se está visitando
        url = str(response.request.url).strip()

        # Cogemos el contenido relevante y para eso debemos usar selectores CSS
        for article in response.css ('.b-page '):

            # Cogemos el contenido del título
            title = str (article.css ('.b-headline__title  ').get ()).strip()
            title = BeautifulSoup (title, 'html.parser').get_text().strip()

            # autor
            autor = str (article.css ('.author.url.fn ').get ()).strip()
            autor = BeautifulSoup (autor, 'html.parser').get_text().strip()

            # Date
            date = str (article.css ('.b-byline__time.date.dtreviewed').get ()).strip()
            date = BeautifulSoup (date, 'html.parser').get_text().strip()

            #Content
            content = "".join (article.css ('.b-content.b-single__content.h-article-content ').get ())
            content = BeautifulSoup (content, 'html.parser').get_text().strip().replace("\"","").replace("\n","")

            data = {
                'url' : url,
                'title': title,
                'autor': autor,
                'time': time,
                'content': content
            }


            title_hash = hashlib.md5(title.encode()).hexdigest()

            filename = f"{title_hash}.json"

            # Guardamos el documento si tiene contenido y título
            if content and title:
                print ("-------------------------")
                print (url)
                print (title)
                print (autor)
                print (time)
                print (content)
                print ("-------------------------")
                self.nDocumentos = self.nDocumentos + 1
                with open ('digitalTrends/' + filename, 'w') as f:
                    json.dump (data, f, indent = 4)



        # Obtenemos todas las otros links de la página representados por la etiqueta <a>


        url_in_current_document = response.css ('a')

        for next_page in url_in_current_document:
            # Para limitar que solamente se parseen las noticias dentro de 'https://www.um.es/web/sala-prensa/'
            # obtenemos el atributo href de la etiqueta <a> y parseamos la página

            url_str = str(next_page.css('::attr(href)').get())


            if ("https://www.digitaltrends.com/computing/" in url_str) and ("/mobile/" in url_str) and url_str not in self.visited_urls and self.nDocumentos < 1500:
              self.visited_urls.add(url_str)
              yield response.follow (next_page, self.parse)

## Apartado 1.2
Para poder lanzar el Spider necesitamos que ejecutar el siguiente código donde se configuará y lanzará el proceso.
Hay que hacer notar que solamente se puede lanzar un proceso por cada sesión en Jupyter notebook es por eso por lo que se recomienda exportar el código en un script de Python .py para poder ejecutarlo desde la línea de comandos.

# Nueva sección

In [ ]:
import os
import scrapy
from scrapy.crawler import CrawlerProcess

# Creamos un proceso de Crawler podemos poner distintas settings que están definidas en la documentación.
# Entre ellas podemos ocular los logs del proceso de Crawling.
process = CrawlerProcess(settings={
    "LOG_ENABLED": False,
    # Used for pipeline 1
})

# Como se ha definido anteriormente en el RSSCrawler, los ficheros se van a almacenar en la carpeta "rss"
# Comprobamos que existe la carpeta y si no existe la creamos
if (not os.path.exists('digitalTrends')):
    os.mkdir('digitalTrends')

# Creamos el proceso con el RSSSpider
process.crawl(DigitalTrendsSpider)
# Ejecutamos el Crawler
process.start()